In [ ]:
week_to_book = 3

## Imports

In [ ]:
import datetime
import time
import zoneinfo

from autoshop import all as autoshop

## Login

In [ ]:
driver = autoshop.chrome.driver()

In [ ]:
autoshop.tesco.login(driver=driver)

## Go to book a slot

In [ ]:
xpath_my_orders = "//a//span[text()='My orders']/.."
autoshop.selenium.wait_and_execute_click(driver=driver, value=xpath_my_orders)

In [ ]:
xpath_book_another_slot = "//a//span[text()='Book a slot']/.."
autoshop.selenium.wait_and_execute_click(driver=driver, value=xpath_book_another_slot)

## Book the slot

### Find next Friday

In [ ]:
now = datetime.datetime.now(tz=zoneinfo.ZoneInfo("Europe/London"))
next_friday = now + datetime.timedelta((4 - now.weekday()) % 7)

autoshop.logger.info(f"{now=:%Y-%m-%d}")
autoshop.logger.info(f"{next_friday=:%Y-%m-%d}")

### Move to the week we are interested in

In [ ]:
autoshop.tesco.go_to_delivery_slots(driver=driver)

for index in range(week_to_book):
    xpath_next_seven_days = "//button//span[text()='Next 7 days']/.."
    autoshop.selenium.wait_and_execute_click(driver=driver, value=xpath_next_seven_days)
    time.sleep(1)

### Find the datetime we want to book for

In [ ]:
booking_date = next_friday + datetime.timedelta(days=week_to_book * 7)
autoshop.logger.info(f"{booking_date=:%Y-%m-%d}")

datetime_slot_start = booking_date.replace(hour=20, minute=0, second=0)
datetime_slot_end = datetime_slot_start + datetime.timedelta(hours=1)

autoshop.logger.info(f"""
{datetime_slot_start=:%Y-%m-%d %H:%M}
{datetime_slot_end=:%Y-%m-%d %H:%M}
""")

time_start = f"{datetime_slot_start:%#I%p}".lower()
time_end = f"{datetime_slot_end:%#I%p}".lower()

xpath_slot_booking_datetime = (
    f"//button[starts-with(@aria-label, '{datetime_slot_start:%A} {datetime_slot_start.day}') "
    f"and contains(@aria-label, ' {datetime_slot_start:%B}, {time_start} – {time_end}')]"
)
autoshop.logger.info(f"""{xpath_slot_booking_datetime=
}""")
autoshop.selenium.wait_and_click(driver=driver, value=xpath_slot_booking_datetime)

### Start Shopping

In [ ]:
xpath_start_shopping = "//span[text()='Start shopping']/.."
autoshop.selenium.wait_and_click(driver=driver, value=xpath_start_shopping)

### Go previous orders and add all to basket

In [ ]:
xpath_my_orders = "//span[text()='My orders']"
autoshop.selenium.wait_and_click(driver=driver, value=xpath_my_orders)
xpath_recent_orders = "//span[text()='Recent']"
autoshop.selenium.wait_and_click(driver=driver, value=xpath_recent_orders)

xpath_add_all_to_basket = (
    "//button[@type='button']//span[text()='Add all to basket']/.."
)
autoshop.selenium.wait_and_click(driver=driver, value=xpath_add_all_to_basket)
time.sleep(10)

## Checkout

In [ ]:
autoshop.tesco.checkout(driver=driver)

In [ ]:
autoshop.tesco.pay(driver=driver)